<a href="https://colab.research.google.com/github/X4651/tibame-gad253/blob/main/tibame_0708_momo%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dirtyjson

In [ ]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)
f = open("data.json", "w", encoding="utf-8")
#                   indent縮排
json.dump(datas, f, indent=4, ensure_ascii=False)
f.close()

In [ ]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)
items = datas["mainEntity"]["itemListElement"]
for item in items:
    item_name = item["name"]
    item_image = item["image"]
    item_url = item["url"]
    item_price = item["offers"]["price"]
    print(item_name)
    print(item_image)
    print(item_url)
    print(item_price)
    # 請再進去每個item的url
    # 取出 介紹/品牌/三個價格

In [ ]:
import urllib.request as req
import bs4 as bs
import json
import dirtyjson

url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())

scripts = html.find_all("script")
target_script = None
for script in scripts:
    if "@type" in script.text:
        target_script = script
        break
datas = dirtyjson.loads(target_script.text)

table = []
items = datas["mainEntity"]["itemListElement"]
for item in items:
    item_name = item["name"]
    item_image = item["image"]
    item_url = item["url"]
    item_price = item["offers"]["price"]
    print(item_name)
    print(item_image)
    print(item_url)
    print(item_price)
    # 你要再進去每個item的url
    # 介紹/品牌/三個價格
    row = get_product_info(item_url)
    row["name"] = item_name
    row["image"] = item_image
    row["url"] = item_url
    table.append(row)
print(table)

In [ ]:
import urllib.request as req
import bs4 as bs

def get_product_info(url):
    # url = "https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7597129&Area=search&mdiv=403&oid=1_7&cid=index&kw=%E5%90%B9%E9%A2%A8%E6%A9%9F"
    resp = req.urlopen(url)
    html = bs.BeautifulSoup(resp.read())

    brand = html.find("a", {"id":"webBrand"})
    infos = html.find("ul", {"id":"categoryActivityInfo"}).find_all("li")
    prices = html.find("ul", {"class":"prdPrice"}).find_all("li")
    if len(prices) == 3:
        price_original = prices[0].find("del")
        price_discount = prices[1].find("del")
        price_special = prices[2].find("span")
    elif len(prices) == 2:
        price_original = prices[0].find("del")
        price_discount = prices[1].find("span")
        price_special = None
    else:
        price_original = None
        price_discount = prices[0].find("span")
        price_special = None


    brand_text = brand.text
    # price
    if not price_original == None:
        price_original_text = price_original.text
    else:
        price_original_text = "-"
    price_discount_text = price_discount.text
    if not price_special == None:
        price_special_text = price_special.text
    else:
        price_special_text = "-"
    # info
    info_text = ""
    for info in infos:
        info_text = info_text + info.text + "\n"

    # print(brand_text)
    # print(price_original_text)
    # print(price_discount_text)
    # print(price_special_text)
    # print(info_text)

    row = {
        "brand":brand_text,
        "price_original":price_original_text,
        "price_discount":price_discount_text,
        "price_special":price_special_text,
        "goods_info":info_text,
    }
    return row

In [ ]:
# 先做一頁 定義成函式 做成迴圈
import urllib.request as req
import bs4 as bs
import json
import dirtyjson
import pandas as pd

# 存一頁
def get_page_products(url):
    # url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0&curPage=1"
    resp = req.urlopen(url)
    html = bs.BeautifulSoup(resp.read())

    scripts = html.find_all("script")
    target_script = None
    for script in scripts:
        if "@type" in script.text:
            target_script = script
            break
    datas = dirtyjson.loads(target_script.text)

    table = []
    items = datas["mainEntity"]["itemListElement"]
    for item in items:
        item_name = item["name"]
        item_image = item["image"]
        item_url = item["url"]
        item_price = item["offers"]["price"]
        print(item_name)
        print(item_image)
        print(item_url)
        print(item_price)
        # 你要再進去每個item的url
        # 介紹/品牌/三個價格
        row = get_product_info(item_url)
        row["name"] = item_name
        row["image"] = item_image
        row["url"] = item_url
        table.append(row)
    return table



total_table = []
for i in range(3):
    page = i + 1
    url = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword=%E5%90%B9%E9%A2%A8%E6%A9%9F&_isFuzzy=0&searchType=1&cateLevel=0&curPage=" + str(page)
    partial = get_page_products(url)
    total_table.extend(partial)
df = pd.DataFrame(total_table)
df.to_csv("momo.csv", encoding="utf-8")